# Create Zipcode Map between cities and counties

In [9]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np
import sets


/Users/along528/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: the sets module is deprecated


## Load zip code data

In [102]:
zipdf = pd.read_csv('/Users/along528/Dropbox/insight/project/datasets/zipcodes/us_postal_codes.csv')
zipdf = zipdf.fillna(-1)
zipdf = zipdf[zipdf['Postal Code']!=-1]
zipdf

,Postal Code,Place Name,State,State Abbreviation,County,Latitude,Longitude,Unnamed: 7
0,210,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
1,211,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
2,212,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
3,213,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
4,214,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
5,215,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
6,401,Pleasantville,New York,NY,-1,41.1381,-73.7847,-1
7,501,Holtsville,New York,NY,Suffolk,40.9223,-72.6371,-1
8,544,Holtsville,New York,NY,Suffolk,40.9223,-72.6371,-1
9,1001,Agawam,Massachusetts,MA,Hampden,42.0702,-72.6227,-1


In [135]:
zipdf[zipdf['Postal Code']==63135]
zipdf[zipdf['Place Name']=='Miami']



,Postal Code,Place Name,State,State Abbreviation,County,Latitude,Longitude,Unnamed: 7
10367,25134,Miami,West Virginia,WV,Kanawha,38.2968,-81.5547,-1
14544,33101,Miami,Florida,FL,Miami-Dade,25.7791,-80.1978,-1
14545,33102,Miami,Florida,FL,Miami-Dade,25.5584,-80.4582,-1
14546,33107,Miami,Florida,FL,Miami-Dade,25.5584,-80.4582,-1
14548,33110,Miami,Florida,FL,Miami-Dade,25.8469,-80.2083,-1
14549,33111,Miami,Florida,FL,Miami-Dade,25.5584,-80.4582,-1
14550,33112,Miami,Florida,FL,-1,25.7964,-80.3849,-1
14551,33114,Miami,Florida,FL,Miami-Dade,25.5584,-80.4582,-1
14552,33116,Miami,Florida,FL,Miami-Dade,25.5584,-80.4582,-1
14554,33121,Miami,Florida,FL,Miami-Dade,25.5584,-80.4582,-1


Fix specific places

In [131]:
test = pd.DataFrame(zipdf) #.ix('Postal Code',43581)
test.ix[test['Postal Code'] == 60018,'Place Name'] = 'Rosemont'
test.ix[test['Postal Code'] == 60176,'Place Name'] = 'Rosemont'
test.ix[test['Postal Code'] == 61554,'Place Name'] = 'Marquette Heights'
test.ix[test['Postal Code'] == 60015,'Place Name'] = 'Riverwoods'
test.ix[test['Postal Code'] == 60118,'Place Name'] = 'East Dundee'
test.ix[test['Postal Code'] == 28445,'Place Name'] = 'Surf City'
test.ix[test['Postal Code'] == 27244,'Place Name'] = 'Elon'
test.ix[test['Postal Code'] == 28584,'Place Name'] = 'Cape Carteret'
test.ix[test['Postal Code'] == 60527,'Place Name'] = 'Burr Ridge'
test.ix[test['Postal Code'] == 61616,'Place Name'] = 'Peoria Heights'
test.ix[test['Postal Code'] == 60073,'Place Name'] = 'Round Lake Heights'
test.ix[test['Postal Code'] == 28227,'Place Name'] = 'Mint Hill'
test.ix[test['Postal Code'] == 6066,'Place Name'] = 'Vernon'
test.ix[test['Postal Code'] == 6340,'Place Name'] = 'Groton Town'
test.ix[test['Postal Code'] == 6349,'Place Name'] = 'Groton Town'
test.ix[test['Postal Code'] == 63135,'Place Name'] = 'Ferguson'
zipdf = test

## Load Police Survey data

In [4]:
dbname = 'police_bjs'
username = 'along528'
pswd = 'password'
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [136]:
surveydf = pd.read_sql('SELECT * FROM police_data_table',con)
#remove state, highway, and public safety departments
surveydf = surveydf[surveydf['agency'].str.contains('STATE')==False]
surveydf = surveydf[surveydf['agency'].str.contains('HIGHWAY')==False]
surveydf = surveydf[surveydf['agency'].str.contains('HWAY')==False]
surveydf = surveydf[surveydf['agency'].str.contains('SAFETY')==False]
surveydf = surveydf[surveydf['agency'].str.contains('PUBLIC')==False]
#fix some city names
surveydf
surveydf.ix[surveydf['city']=='New York','city']='New York City'
surveydf.ix[surveydf['city']=='Roxbury Crossing','city']='Boston'
surveydf.ix[(surveydf['city']=='Doral') & (surveydf['state']=='FL'),'city']='Miami'
#surveydf

Make sure I also look at counties

After some specific fixes, there are 5 cities in CT, NC and IL from the survey data that don't match at least one zipcode

In [104]:

zipmap = {}
count = 0
for index,survey_series in surveydf.iterrows():
    survey_name = survey_series['agency']
    survey_city = survey_series['city']
    survey_state = survey_series['state']
    #if 'sherif' in survey_name.lower() or 'county' in survey_name.lower(): continue
    if survey_state!='NC' and survey_state!='IL' and survey_state!='CT': continue
    
    #if survey_state!='NC': continue # and survey_state!='IL' and survey_state!='CT': continue

    testdf = zipdf[(zipdf['Place Name']==survey_city) & (zipdf['State Abbreviation']==survey_state)]
    if survey_state not in zipmap:
        #sets_map[survey_state] = sets.Set(testdf['Postal Code'].tolist())
        zipmap[survey_state] = []
        zipmap[survey_state]+=testdf['Postal Code'].tolist()

    else:
        #sets_map[survey_state].union(sets.Set(testdf['Postal Code'].tolist()))
        zipmap[survey_state]+=testdf['Postal Code'].tolist()

    if len(testdf)==0: 
        print survey_name,survey_city,survey_state
        count+=1
    #print survey_name,survey_city,survey_state,len(testdf)
print count

SAUGET POLICE DEPT Sauget IL
WILLOWBROOK POLICE DEPT Willowbrook IL
ROUND LAKE POLICE DEPARTMENT Round Lake IL
LAUREL PARK POLICE DEPARTMENT Laurel Park NC
WHISPERING PINES POLICE DEPARTMENT Whispering Pines NC
5


In [30]:
print len(zipmap['CT'])
print len(list(sets.Set(zipmap['CT'])))


166
166


Those zipcodes in NC, CT and IL that have not been matched are

In [108]:
zip_traffic_states = zipdf[(zipdf['State Abbreviation']=='NC') |\
                           (zipdf['State Abbreviation']=='CT') |\
                           (zipdf['State Abbreviation']=='IL')]
unmatched_zip_traffic_states = zip_traffic_states[zip_traffic_states['Postal Code'].isin(list(sets.Set(zipmap['CT']))+\
                                                          list(sets.Set(zipmap['IL']))+\
                                                          list(sets.Set(zipmap['NC'])))==False]
unmatched_zip_traffic_states.to_csv('unmatched_zip_ct_il_nc.csv')

The most glaring omission from the survey data appears to beNew Haven, CT.
Most of the rest seem to be small (i.e. single zipcode) towns that I've never heard of.

As for all states, 160 in total are ummatched

In [137]:

zips = []
count = 0
for index,survey_series in surveydf.iterrows():
    survey_name = survey_series['agency']
    survey_city = survey_series['city']
    survey_state = survey_series['state']
    #if 'sherif' in survey_name.lower() or 'county' in survey_name.lower(): continue
    #if survey_state!='NC' and survey_state!='IL' and survey_state!='CT': continue
    
    #if survey_state!='NC': continue # and survey_state!='IL' and survey_state!='CT': continue

    testdf = zipdf[(zipdf['Place Name']==survey_city) & (zipdf['State Abbreviation']==survey_state)]

    zips+=testdf['Postal Code'].tolist()

    if len(testdf)==0: 
        print survey_name,survey_city,survey_state
        count+=1
    #print survey_name,survey_city,survey_state,len(testdf)
print count

HOOVER POLICE DEPARTMENT Hoover AL
HOMEWOOD POLICE DEPT Homewood AL
MIDFIELD POLICE DEPT Midfield AL
VERNON POLICE DEPARTMENT Vernon CA
HOOPA TRIBAL POLICE DEPARTMENT   CA
CHERRY HILLS VILLAGE POLICE DEPT Cherry Hills Village CO
THORNTON POLICE DEPARTMENT Thornton CO
LAKEWOOD POLICE DEPARTMENT Lakewood CO
ELSMERE BUREAU OF POLICE Elsmere DE
LAUDERHILL POLICE DEPARTMENT Lauderhill FL
SOUTH MIAMI POLICE DEPARTMENT South Miami FL
MIAMI SPRINGS POLICE DEPARTMENT Miami Springs FL
SOUTH DAYTONA POLICE DEPARTMENT South Daytona FL
RIVIERA BEACH POLICE DEPARTMENT Riviera Beach FL
CORAL SPRINGS POLICE DEPARTMENT Coral Springs FL
SUNRISE POLICE DEPARTMENT Sunrise FL
PLANTATION POLICE DEPARTMENT Plantation FL
MIRAMAR POLICE DEPARTMENT Miramar FL
MARGATE POLICE DEPARTMENT Margate FL
DAVIE POLICE DEPARTMENT Davie FL
CORAL GABLES POLICE DEPT Coral Gables FL
SURFSIDE POLICE DEPARTMENT Surfside FL
COCONUT CREEK POLICE DEPARTMENT Coconut Creek FL
DAYTONA BEACH SHORES DPS Daytona Beach Shores FL
PORT ST 

In [123]:
unmatched_zip_all_states = zipdf[zipdf['Postal Code'].isin(list(sets.Set(zips)))==False]
unmatched_zip_all_states.to_csv('unmatched_zip_all_states.csv')

Had to fix some mismatches that didn't find Boston, New York, Ferguson, Miami

The glaring mismatches remaining after some editing:
    

MIAMI-DADE POLICE DEPARTMENT Doral FL


